In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import math
import warnings
warnings.filterwarnings("ignore")


### This routine handles these steps:
#### 1) read in projection data
#### 2) merge with population data for each group
#### 3) converge age groups to total population
#### 4) get update population shares
#### 5) read into gams , run gams 
#### 6) re-import gams results (housing prices, wages, and updates population shares (from logit)
#### 7) prep gams results for incorporation into logit routine
#### 8) re-run shares in stata
#### 9) map out migration flows for top entry exit


### Steps 1-5
##### (1) read in projection data, (2) merge pop data fro each group, (3) translate age group and skill changes to total population pct change (4) get updated gams shares (5) read into gams, and run new counter factual 1 time

In [2]:
# tmp0_gr=tmp0.groupby(['msa','statefip'],as_index=False).agg({'id':sum})
# bmk0_gr=bmk0.groupby(['msa'],as_index=False).agg({'id':sum})
# msaid_state_lookup=tmp0_gr[['msa','statefip']].merge(bmk0_gr[['msa']],on='msa').groupby(['msa','statefip'],as_index=False).sum()
# msaid_state_lookup.to_excel('/Users/hannahkamen/Downloads/msaid_state_lookup.xlsx')



In [3]:
#new_bmk=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/projection_data_age2_1_wbmk.dta')
msa=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/projection_data_age2_%s_wbmk_iter1.dta'%i)
            
    

In [4]:
###these are the population shares changes from the first cycle of GAMS data inputs (after initial logit shock with temp)
###the shares caluclated here represent the population change from the previous iteration 
state_split=pd.read_excel('/Users/hannahkamen/Downloads/state_pop_educ_shares.xlsx')
state_age_shares=pd.read_excel('/Users/hannahkamen/Downloads/state_age_shares.xlsx')
msaid_state_lookup=pd.read_excel('/Users/hannahkamen/Downloads/msaid_state_lookup.xlsx')

state_split=state_split[['skl_2','skl_3','skl_4','skl_5','skl_6','skl_7','unskl_2','unskl_3','unskl_4','unskl_5','unskl_6','unskl_7','state','state_pop']]

master=pd.DataFrame()
for a in [2]:
    for i in [0,1]:       
        try:
            del tmp0
        except:
            print("first loop")
        
        #(1) read in projection data, and benchmark population shares

        tmp0=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/projection_data_age2_%s_wbmk_iter1.dta'%i)
        
        #############merge state lookup onto both projections 
        #sl=pd.read_excel('/Users/hannahkamen/Downloads/statelookup2.xlsx')
        #tmp0=tmp0.merge(sl,on='statefip',how='inner')

        #(2) merge pop share estimates from each group

        ########group benchmark and projected by state, sum shares and chosen to get pct change
        tmp=tmp0.groupby(['state'],as_index=False).agg({'fexthot_28':max,'fextcold':max,'share_it1':sum,'share':sum,'share_b':sum})

        ########merge grouped projections with benchmark to get pct change
        tmp['pct_change']=(tmp['share_it1']-tmp['share'])/tmp['share']

        ########merge pct changes onto population by age

        tmp=tmp.merge(state_age_shares,on='state',how='inner')
        tmp['age_id']= a
        tmp['educ_id']=i
        ########SET AGE GROUP TWO TO 100 PERCENT OF POPULATION FOR NOW
        #tmp['contribution_to_total_change']=tmp['pct_change']*tmp['%s'%a]
        tmp['contribution_to_total_change']=tmp['pct_change']*1
        
        ######## append all age and skill datasets
        master=master.append(tmp)






first loop


In [55]:
list(tmp0)

['id',
 'msa',
 'bpl',
 'chosen',
 'd_s',
 'd_r1',
 'd_r2',
 'inc_hat',
 'coll',
 'hot_26C_ca',
 'hot_28C_ca',
 'hot_30C_ca',
 'cold_0C_ca',
 'hot_26C_w',
 'hot_28C_w',
 'hot_30C_w',
 'cold_0C_w',
 'hot_26C_mw',
 'hot_28C_mw',
 'hot_30C_mw',
 'cold_0C_mw',
 'hot_26C_ne',
 'hot_28C_ne',
 'hot_30C_ne',
 'cold_0C_ne',
 'cold_0C_coll',
 'hot_26C_coll',
 'hot_28C_coll',
 'hot_30C_coll',
 'hot_mm_ca',
 'cold_mm_ca',
 'hot_mm_w',
 'cold_mm_w',
 'hot_mm_mw',
 'cold_mm_mw',
 'hot_mm_ne',
 'cold_mm_ne',
 'hot_mm_coll',
 'cold_mm_coll',
 'hot_sw_ca',
 'cold_sw_ca',
 'hot_sw_w',
 'cold_sw_w',
 'hot_sw_mw',
 'cold_sw_mw',
 'hot_sw_ne',
 'cold_sw_ne',
 'hot_sw_coll',
 'cold_sw_coll',
 'd_s_kid',
 'd_r1_kid',
 'd_r2_kid',
 'index',
 'regionfe',
 'statefip',
 'lnrho',
 'metarea',
 'hot',
 'cold',
 'avgtemp',
 'precip',
 'dewpt',
 'relhum',
 'sun',
 'lnsea',
 'lnsea2',
 'lnlake',
 'lnlake2',
 'lnslope',
 'lnden',
 'lnhsgrad',
 'lncoll',
 'lngrad',
 'lnage',
 'lnhisp',
 'lnblack',
 'lnprec',
 'lndewpt',

In [53]:
len(list(tmp0['share_msa_b'].unique()))

258

In [44]:
tmp0['msa_pct_pop_change'].unique()

array([-4.63626608e-02, -1.13919385e-01, -3.16062793e-02, -1.70702115e-01,
       -8.46888870e-02, -4.27398272e-02,  1.17279053e-01,  3.47240157e-02,
       -4.73078676e-02, -8.77993107e-02, -2.95080394e-02, -3.14616531e-01,
        1.14578940e-01, -6.63982406e-02,  3.51601187e-03, -1.89047039e-01,
        1.17727909e-02,  1.96995094e-01, -1.01829320e-01, -1.42851723e-02,
        3.01442277e-02,  1.08045630e-01,  2.37697244e-01,             nan,
       -8.91612917e-02,  1.63827121e-01,  6.56571891e-03,  6.86960667e-02,
       -6.36901855e-02, -3.80600803e-02, -2.33106539e-01,  6.48042560e-02,
       -2.97715813e-01,  5.37317581e-02,  1.30240932e-01,  3.06048661e-01,
       -4.35934439e-02, -6.69911951e-02,  1.00060582e-01, -1.91060483e-01,
        6.38362393e-02, -5.49708977e-02, -4.59890813e-02, -5.69762774e-02,
        8.93479288e-02, -3.51365775e-01, -3.28388244e-01,  1.19310893e-01,
        1.44825205e-01, -1.03105806e-01, -6.28904477e-02,  2.43737102e-02,
        1.56758368e-04, -

In [42]:
list(master)

['state',
 'fexthot_28',
 'fextcold',
 'share_it1',
 'share',
 'pct_change',
 'Unnamed: 0',
 'statefip',
 '0',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 'state_pop',
 'age_id',
 'educ_id',
 'contribution_to_total_change']

In [5]:

#(3) translate age group to total population pct change


master_gr=master.groupby(['state','educ_id'],as_index=False).agg({'contribution_to_total_change':sum})


#####get MSA Identifier information

msa_id=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/msa_identifier.dta')
msa_vars=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/second_stage_dataset_cl.dta')




In [70]:
msa_vars_lm=msa_vars[['msa', 'lnpop','lnrho']]

In [71]:
list(msa_vars_lm)

['msa', 'lnpop', 'lnrho']

In [72]:
msa_vars.to_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/second_stage_dataset_cl1.dta')


In [57]:
list(msa_vars)

['index',
 'wa_fbin158',
 'wa_fbin159',
 'wa_fbin160',
 'wa_fbin161',
 'wa_fbin162',
 'wa_fbin163',
 'wa_fbin164',
 'wa_fbin165',
 'wa_fbin166',
 'wa_fbin167',
 'wa_fbin168',
 'wa_fbin169',
 'wa_fbin170',
 'wa_fbin171',
 'wa_fbin172',
 'wa_fbin173',
 'wa_fbin174',
 'wa_fbin175',
 'wa_fbin176',
 'wa_fbin177',
 'wa_fbin178',
 'wa_fbin179',
 'wa_fbin180',
 'wa_fbin181',
 'wa_fbin182',
 'wa_fbin183',
 'wa_fbin184',
 'wa_fbin185',
 'wa_fbin186',
 'wa_fbin187',
 'wa_fbin188',
 'wa_fbin189',
 'wa_fbin190',
 'wa_fbin191',
 'wa_fbin192',
 'wa_fbin193',
 'wa_fbin194',
 'wa_fbin195',
 'wa_fbin196',
 'wa_fbin197',
 'wa_fbin198',
 'wa_fbin199',
 'wa_fbin200',
 'wa_fbin201',
 'wa_fbin202',
 'wa_fbin203',
 'wa_fbin204',
 'wa_fbin205',
 'wa_fbin206',
 'wa_fbin207',
 'wa_fbin208',
 'wa_fbin209',
 'wa_fbin210',
 'wa_fbin211',
 'wa_fbin212',
 'wa_fbin213',
 'wa_fbin214',
 'wa_fbin215',
 'wa_fbin216',
 'wa_fbin217',
 'wa_fbin218',
 'wa_fbin219',
 'wa_fbin220',
 'wa_fbin221',
 'wa_fbin222',
 'wa_fbin1',
 '

In [6]:
###import state lookup

sl=pd.read_excel('/Users/hannahkamen/Downloads/statelookup2.xlsx')

In [7]:
#(4) export to gams format
master_gr['educ_id']=master_gr['educ_id'].astype(str).str.replace('0','unskl').str.replace('1','skl')
master_gr=master_gr.merge(sl,on='state',how='inner')
master_gr=master_gr.rename(columns={'abbrev':'','educ_id':'sk','contribution_to_total_change':'skill_shr'})


In [8]:
master_gr_lm=master_gr[['','sk','skill_shr']]
master_gr_lm.to_csv('/Users/hannahkamen/Downloads/le0_shock_0_it1.csv',index=False)

In [9]:
master_gr['skill_shr'].describe()

count    88.000000
mean     -0.017610
std       0.111226
min      -0.327351
25%      -0.071204
50%      -0.005141
75%       0.040509
max       0.247840
Name: skill_shr, dtype: float64

In [ ]:
master_gr['skill_shr'].describe()

In [ ]:
#tmp['fexthot_28'].describe()
#tmp['fextcold'].describe()

In [ ]:
npl_rpt['value'].max()

#### (6) re-import GAMS results, merge with pop changes

In [35]:
##### import GAMS results
#y_rpt=pd.read_csv('/Users/hannahkamen/Downloads/y_rpt.csv')
phou_rpt=pd.read_csv('/Users/hannahkamen/Downloads/phou_rpt1.csv')
npl_rpt=pd.read_csv('/Users/hannahkamen/Downloads/npl_rpt1.csv')
####msa pop data
msa=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/second_stage_dataset_cl.dta')

sl=pd.read_excel('/Users/hannahkamen/Downloads/statelookup2.xlsx')


In [47]:
test_wage=pd.read_csv('/Users/hannahkamen/Downloads/npl_rpt.csv')
test_ph=pd.read_csv('/Users/hannahkamen/Downloads/phou_rpt.csv')

In [29]:
test_wage[test_wage['region']=='MT']

,file,pct,region,skill,value
92,staticmodel_v7_.05,0.05,MT,skl,1.003403
93,staticmodel_v7_.05,0.05,MT,unskl,1.084297


In [11]:
####create table with percent of total state that each MSA contributes to
# msa=msa[['statefip','msa','lnpop']]
# msa['msa_pop']=np.exp(msa['lnpop'])
# msa_tot=msa.groupby('statefip',as_index=False).agg({'msa_pop':sum})
# msa_tot=msa_tot.rename(columns={'msa_pop':'msa_pop_total'})
# msa=msa.merge(msa_tot,on='statefip')
# msa['pct_state_total']=msa['msa_pop']/msa['msa_pop_total']
# msa.to_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/msa_pop_pct.dta')


In [36]:
npl_rpt=npl_rpt[['region','skill','value']]
npl_rpt=npl_rpt.rename(columns={'region':'abbrev','value':'pl'})

In [37]:
phou_rpt=phou_rpt[['pct','region']]
phou_rpt=phou_rpt.rename(columns={'region':'ph','pct':'abbrev'})

In [38]:
#####import state pop shares by skilled and unskilled
state_educ=pd.read_excel('/Users/hannahkamen/Downloads/state_educ_shares.xlsx')
del state_educ['statefip']
state_educ=state_educ.merge(sl,on='state',how='inner')
state_educ=state_educ[['statefip','skl','unskl','state','abbrev','state_pop']]

In [39]:
list(master_gr)

['', 'sk', 'skill_shr']

In [40]:
###merge all fields
r_df=phou_rpt.merge(npl_rpt, on='abbrev').merge(master_gr, left_on=['abbrev','skill'], right_on=['','sk']).merge(state_educ,on='abbrev')



In [41]:
list(r_df)

['abbrev',
 'ph',
 'skill',
 'pl',
 '',
 'sk',
 'skill_shr',
 'statefip',
 'skl',
 'unskl',
 'state',
 'state_pop']

In [18]:
####pivot skill pct changes
pop_changes=r_df.pivot(index='state',columns='sk',values='skill_shr').reset_index()
pop_changes=pop_changes.rename(columns={'skl':'skl_pct_delta','unskl':'unskl_pct_delta'})

#####pivot labor prices
pl_changes=r_df.pivot(index='state',columns='sk',values='pl').reset_index()
pl_changes=pl_changes.rename(columns={'skl':'pl_skl','unskl':'pl_unskl'})

In [19]:
#r_df=r_df[['statefip','abbrev','state','skill','pl','ph','skl','unskl','state_pop']]
r_df=r_df.merge(pop_changes,on='state').merge(pl_changes,on='state')
r_df['skilled_level_change']=(r_df['skl']*r_df['state_pop']) + (r_df['skl']*r_df['state_pop']*r_df['skl_pct_delta'])
r_df['unskilled_level_change']=(r_df['unskl']*r_df['state_pop']) + (r_df['unskl']*r_df['state_pop']*r_df['unskl_pct_delta'])
r_df['new_state_pop']=r_df['unskilled_level_change']+r_df['skilled_level_change']



In [20]:
r_df_lm=r_df.drop_duplicates(subset='state')

In [24]:
master_gr['skill_shr'].min()

-0.32735109329223633

In [33]:
r_df_lm['skl_pct_delta'].min()

-0.32735109329223633

In [34]:
r_df_lm.to_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/gams_dta1.dta')




In [73]:
list(r_df_lm)

['statefip',
 'abbrev',
 'state',
 'skill',
 'pl',
 'ph',
 'skl',
 'unskl',
 'state_pop',
 'skl_pct_delta',
 'unskl_pct_delta',
 'pl_skl',
 'pl_unskl',
 'skilled_level_change',
 'unskilled_level_change',
 'new_state_pop']

In [49]:
test_ph.head()

,file,pct,region,sector,value
0,staticmodel_v7_.05,IA,0.996380,NaN,NaN
1,staticmodel_v7_.05,FL,0.991287,NaN,NaN
2,staticmodel_v7_.05,MD,1.001167,NaN,NaN
3,staticmodel_v7_.05,OH,0.998414,NaN,NaN
4,staticmodel_v7_.05,TN,0.999079,NaN,NaN


In [50]:
test_ph[test_ph['pct']=='ID']

,file,pct,region,sector,value
43,staticmodel_v7_.05,ID,0.992619,NaN,NaN


In [45]:
r_df_lm[r_df_lm['state']=='idaho']

,statefip,abbrev,state,skill,pl,ph,skl,unskl,state_pop,skl_pct_delta,unskl_pct_delta,pl_skl,pl_unskl,skilled_level_change,unskilled_level_change,new_state_pop
76,16,ID,idaho,skl,1.037671,0.987707,0.387703,0.612297,1567582,-0.241002,-0.297493,1.037671,1.193085,461286.155059,674284.32556,1.135570e+06


In [ ]:
r_df_lm[r_df_lm['state']=='arizona']['ph'].astype(float)*10000

In [ ]:
####translates pop changes by skill into total population changes


In [ ]:
np.exp(tmp0['inc_hat'])

##### (9) map out migration flows

In [ ]:
#get sum of shares in all other states for people who chose states 1-44
#get ids that live in state s currentlly
master_map=pd.DataFrame()
for s in df_m['state'].unique():
    living_now=[]
    df_lm=df_m[(df_m['state']==s) & (df_m['chosen']==1)]
    ####list of ids currently living in state s
    living_now.append(list(df_lm['id'].unique())[0])
    ###limit dataframe to the ids of people living in state s
    tmp=df_m[df_m['id'].isin(living_now)]
    ####now groupby shares across all states
    tmp2=tmp.groupby(['state'],as_index=False).agg({'share':sum})
    ###tag origin state
    tmp2['living_flag']=s
    master_map=master_map.append(tmp2)
    
######merge with state populaiton
master_map0=master_map.merge(state_split,left_on='living_flag',right_on='state',how='inner')
master_map0=master_map0.rename(columns={'state_x':'moving_to'})

####get estimates for number of people leaving / going to
master_map0['arrive_moving_to']=master_map0['unskl_2']*master_map0['state_pop']

####merge in the msa codes

master_map0_lm=master_map0[master_map0['living_flag']=='texas']

###export texas test

master_map0_lm.to_csv('/Users/hannahkamen/Downloads/texas_out_unskl2.csv')
    
    

In [ ]:
map_dta=pd.read_csv('/Users/hannahkamen/Downloads/census_texas.csv')

In [ ]:
map_dta_lm=map_dta[map_dta['GEOID2'].str.contains(',')]

map_dta_lm['state']=map_dta_lm['GEOID2'].apply(lambda x: x.split(',')[1])
map_dta_lm['state']=map_dta_lm['GEOID2'].apply(lambda x: x.split(',')[1])

In [ ]:
map_dta_lm

In [ ]:
map_dta['state']

In [ ]:
list(map_dta['GEOID2'])

In [ ]:
master_map0.head()

In [ ]:
##test to see what is happening with temperature

In [ ]:
# df_tmp[df_tmp['state']=='washington']

In [ ]:
master_map.sort_values(by='share',ascending=False).head(100)

In [ ]:
master_map.head()

In [ ]:
df_lm

In [ ]:
master_map.head()

In [ ]:
df_test_gre

In [ ]:
df_tmp

In [ ]:
df_test_gre

In [ ]:
# df_m[(df_m['id']==3)& (df_m['state']=='california')][['msa','bpl','chosen','d_s','d_r1','d_r2','statefip','state']]




In [ ]:
df_test_gre['diff']=abs(df_test_gre['share']-df_test_gre['chosen'])

In [ ]:
df_test_gre['diff'].describe()

In [ ]:
# df_tmp=df_m.groupby(['msa'],as_index=False).agg({'fexthot_28':sum,'fextcold':sum,'state':max,'hot':sum,'cold':sum})

In [ ]:
# df_m[['msa','state','hot','cold','fexthot_28','fextcold']].head()

In [ ]:
len(df[(df['id']==4100)]['msa'].unique())

In [ ]:
list()

In [ ]:
logit_ready[logit_ready['chosen']==1]['d_s'].describe()

In [ ]:
df_lm=df[['msa','chosen','share']]

In [ ]:
df[df['chosen']==1]['share'].describe()

#### Import total population and process percent changes for Windc counterfactual

In [ ]:
#orig=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/dta/acs5yr_0610_clr.dta')

#orig.groupby(['ageid'],as_index=False)

In [ ]:
###get shares by college graduates#impo
#educ_shares.to_csv('/Users/hannahkamen/Downloads/le0_shr2.csv',index=False)
# df['pct_col']=np.exp(df['lncoll'])
# df['pct_nc']=1-np.exp(df['lncoll'])
# df['coll_pop']=np.exp(df['lnpop'])*df['pct_col']
# df['population']=np.exp(df['lnpop'])

# df_tot=df.groupby(['statefip'],as_index=False).agg({'population':sum,'coll_pop':sum,'share':sum,'chosen':sum})
# df_tot['state_coll_shr']=df_tot['coll_pop']/df_tot['population']
# df_tot['pct_change']=(df_tot['share']-df_tot['chosen'])/df_tot['chosen']


In [ ]:
df[df['share'].isnull()]

In [ ]:
df

In [ ]:
list(df)

In [ ]:


df_st=df.groupby('statefip',as_index=False).agg({'share':sum,'population':sum,'chosen':sum,'id':'count'})
df_st['chosen'].sum()

In [ ]:
list(df)

In [ ]:
len(df['id'].unique())

In [ ]:
df_st.head()

In [ ]:
len(df_st['id'].unique())

In [ ]:
44*265

In [ ]:
df[df['id']==62973.0]

In [ ]:
len(df)

In [ ]:
df_st.head()

In [ ]:
len(df['statefip'].unique())

In [ ]:
df.groupby('id',as_index=False).agg({'share':sum})['share'].unique()

In [ ]:
df['msa'].unique()

In [ ]:
len(list(df['msa'].unique()))

In [ ]:
df.head()

In [ ]:
data.head()

In [ ]:
st_lkup=data.groupby(['statefip','str_statefip'],as_index=False).sum()[['statefip','str_statefip']]

In [ ]:
list(data)

In [ ]:
data['str_statefip'].unique()

In [ ]:
list(data)

In [ ]:
st_lkup

In [ ]:
data.head()

In [ ]:
len(data)

In [ ]:
list(data)

In [ ]:
len(logit)

In [ ]:
list(logit)

In [ ]:
logit.head()

In [ ]:
q

In [ ]:
logit.head()

In [ ]:
q.head()

In [ ]:
for i in np.arange(2,8,1):
    print(i)

In [ ]:
for i in np.arange(2,8,1):
    df=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/results/temp/2nd_stage_avg_age%s.dta'%i)
    




In [ ]:
df

In [ ]:
secondndstage2.head()

In [ ]:
secondndstage3.head()

In [ ]:
logit['_b_d_r1'].unique()

In [ ]:
logit['d_s'].unique()

In [ ]:
len(cols)

In [ ]:
cols=[x for x in logit.columns if "_b" in x]

In [ ]:
for x in [x for x in logit.columns if "_b" in x]:
    
    print(logit[x].unique())

In [ ]:
logit['d_s']